In [ ]:
import pandas as pd

In [ ]:
df=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-10.parquet")

In [ ]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
count,4.428699e+06,4428699,4428699,3.437812e+06,4.428699e+06,3.437812e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,4.428699e+06,3.437812e+06,3.437812e+06,4.428699e+06
mean,1.881417e+00,2025-10-16 20:07:06.179353,2025-10-16 20:25:46.199610,1.274813e+00,6.696744e+00,4.055683e+00,1.619942e+02,1.616615e+02,9.417603e-01,1.825555e+01,1.175575e+00,4.778671e-01,2.990985e+00,5.307759e-01,9.449223e-01,2.697358e+01,2.173596e+00,1.469934e-01,5.355713e-01
min,1.000000e+00,2025-09-30 22:54:51,2025-09-30 23:04:41,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-8.000000e+02,-7.500000e+00,-5.000000e-01,-9.000000e+01,-9.500000e+01,-1.000000e+00,-8.010000e+02,-2.500000e+00,-1.750000e+00,-7.500000e-01
25%,2.000000e+00,2025-10-09 08:20:44,2025-10-09 08:40:08.500000,1.000000e+00,1.050000e+00,1.000000e+00,1.170000e+02,1.080000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.554000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,2025-10-16 20:16:54,2025-10-16 20:35:38,1.000000e+00,1.890000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,1.350000e+01,0.000000e+00,5.000000e-01,2.220000e+00,0.000000e+00,1.000000e+00,2.145000e+01,2.500000e+00,0.000000e+00,7.500000e-01
75%,2.000000e+00,2025-10-24 14:53:38,2025-10-24 15:16:35,1.000000e+00,3.840000e+00,1.000000e+00,2.330000e+02,2.340000e+02,1.000000e+00,2.261000e+01,2.500000e+00,5.000000e-01,4.110000e+00,0.000000e+00,1.000000e+00,3.114000e+01,2.500000e+00,0.000000e+00,7.500000e-01
max,7.000000e+00,2025-11-01 00:32:12,2025-11-03 09:38:20,9.000000e+00,2.763335e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.071900e+03,2.369000e+01,1.050000e+01,5.500000e+02,1.411800e+02,1.000000e+00,1.104590e+03,2.500000e+00,6.750000e+00,7.500000e-01
std,7.602906e-01,NaN,NaN,6.913772e-01,6.569620e+02,1.673265e+01,6.607760e+01,7.043272e+01,7.244619e-01,1.968846e+01,1.823128e+00,1.344743e-01,4.080894e+00,2.169216e+00,2.875662e-01,2.409999e+01,9.511162e-01,5.321739e-01,3.555875e-01


In [ ]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [ ]:
df[categorical]=df[categorical].astype(str)

In [ ]:
train_dict=df[categorical + numerical].to_dict(orient='records')

: 

In [ ]:
dv=DictVectorizer()
X_train=dv.fit_transform(train_dict)

: 